# Next Word Prediction:

### Importing The Required Libraries:

In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
import pickle
import numpy as np
import os

In [2]:
"""
    Dataset: http://www.gutenberg.org/cache/epub/5200/pg5200.txt
    Remove all the unnecessary data and label it as Metamorphosis-clean.
    The starting and ending lines should be as follows.

"""


file = open("metamorphosis_clean.txt", "r", encoding = "utf8")
lines = []

for i in file:
    lines.append(i)
    
print("The First Line: ", lines[0])
print("The Last Line: ", lines[-1])

The First Line:  ﻿One morning, when Gregor Samsa woke from troubled dreams, he found

The Last Line:  first to get up and stretch out her young body.


### Cleaning the data:

In [3]:
data = ""

for i in lines:
    data = ' '. join(lines)
    
data = data.replace('\n', '').replace('\r', '').replace('\ufeff', '')
data[:360]

'One morning, when Gregor Samsa woke from troubled dreams, he found himself transformed in his bed into a horrible vermin.  He lay on his armour-like back, and if he lifted his head a little he could see his brown belly, slightly domed and divided by arches into stiff sections.  The bedding was hardly able to cover it and seemed ready to slide off any moment.'

In [4]:
import string

translator = str.maketrans(string.punctuation, ' '*len(string.punctuation)) #map punctuation to space
new_data = data.translate(translator)

new_data[:500]

'One morning  when Gregor Samsa woke from troubled dreams  he found himself transformed in his bed into a horrible vermin   He lay on his armour like back  and if he lifted his head a little he could see his brown belly  slightly domed and divided by arches into stiff sections   The bedding was hardly able to cover it and seemed ready to slide off any moment   His many legs  pitifully thin compared with the size of the rest of him  waved about helplessly as he looked    What s happened to me   he'

In [5]:
z = []

for i in data.split():
    if i not in z:
        z.append(i)
        
data = ' '.join(z)
data[:500]

'One morning, when Gregor Samsa woke from troubled dreams, he found himself transformed in his bed into a horrible vermin. He lay on armour-like back, and if lifted head little could see brown belly, slightly domed divided by arches stiff sections. The bedding was hardly able to cover it seemed ready slide off any moment. His many legs, pitifully thin compared with the size of rest him, waved about helplessly as looked. "What\'s happened me?" thought. It wasn\'t dream. room, proper human room altho'

### Tokenization:

In [6]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts([data])

# saving the tokenizer for predict function.
pickle.dump(tokenizer, open('tokenizer1.pkl', 'wb'))

sequence_data = tokenizer.texts_to_sequences([data])[0]
sequence_data[:10]

[17, 53, 293, 2, 18, 729, 135, 730, 294, 8]

In [7]:
vocab_size = len(tokenizer.word_index) + 1
print(vocab_size)

2617


In [8]:
sequences = []

for i in range(1, len(sequence_data)):
    words = sequence_data[i-1:i+1]
    sequences.append(words)
    
print("The Length of sequences are: ", len(sequences))
sequences = np.array(sequences)
sequences[:10]

The Length of sequences are:  3889


array([[ 17,  53],
       [ 53, 293],
       [293,   2],
       [  2,  18],
       [ 18, 729],
       [729, 135],
       [135, 730],
       [730, 294],
       [294,   8],
       [  8, 731]])

In [9]:
X = []
y = []

for i in sequences:
    X.append(i[0])
    y.append(i[1])
    
X = np.array(X)
y = np.array(y)

In [10]:
print("The Data is: ", X[:5])
print("The responses are: ", y[:5])

The Data is:  [ 17  53 293   2  18]
The responses are:  [ 53 293   2  18 729]


In [11]:
y = to_categorical(y, num_classes=vocab_size)
y[:5]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

### Creating the Model:

In [12]:
model = Sequential()
model.add(Embedding(vocab_size, 10, input_length=1))
model.add(LSTM(1000, return_sequences=True))
model.add(LSTM(1000))
model.add(Dense(1000, activation="relu"))
model.add(Dense(vocab_size, activation="softmax"))

In [13]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 1, 10)             26170     
                                                                 
 lstm (LSTM)                 (None, 1, 1000)           4044000   
                                                                 
 lstm_1 (LSTM)               (None, 1000)              8004000   
                                                                 
 dense (Dense)               (None, 1000)              1001000   
                                                                 
 dense_1 (Dense)             (None, 2617)              2619617   
                                                                 
Total params: 15,694,787
Trainable params: 15,694,787
Non-trainable params: 0
_________________________________________________________________


### Plot The Model:

In [14]:
from tensorflow import keras
from keras.utils.vis_utils import plot_model

keras.utils.plot_model(model, to_file='model.png', show_layer_names=True)

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model/model_to_dot to work.


### Callbacks:

In [15]:
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.callbacks import TensorBoard

checkpoint = ModelCheckpoint("nextword1.h5", monitor='loss', verbose=1,
    save_best_only=True, mode='auto')

reduce = ReduceLROnPlateau(monitor='loss', factor=0.2, patience=3, min_lr=0.0001, verbose = 1)

logdir='logsnextword1'
tensorboard_Visualization = TensorBoard(log_dir=logdir)

### Compile The Model:

In [16]:
model.compile(loss="categorical_crossentropy", optimizer=Adam(lr=0.001))

C:\Users\praso\anaconda3\envs\all\lib\site-packages\keras\optimizer_v2\adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


### Fit The Model:

In [17]:
model.fit(X, y, epochs=150, batch_size=64, callbacks=[checkpoint, reduce, tensorboard_Visualization])

Epoch 1/150
61/61 [==============================] - ETA: 0s - loss: 7.8751
Epoch 1: loss improved from inf to 7.87515, saving model to nextword1.h5
61/61 [==============================] - 12s 66ms/step - loss: 7.8751 - lr: 0.0010
Epoch 2/150
61/61 [==============================] - ETA: 0s - loss: 7.8632
Epoch 2: loss improved from 7.87515 to 7.86319, saving model to nextword1.h5
61/61 [==============================] - 4s 65ms/step - loss: 7.8632 - lr: 0.0010
Epoch 3/150
60/61 [============================>.] - ETA: 0s - loss: 7.8278
Epoch 3: loss improved from 7.86319 to 7.82783, saving model to nextword1.h5
61/61 [==============================] - 4s 66ms/step - loss: 7.8278 - lr: 0.0010
Epoch 4/150
61/61 [==============================] - ETA: 0s - loss: 7.6444
Epoch 4: loss improved from 7.82783 to 7.64436, saving model to nextword1.h5
61/61 [==============================] - 4s 66ms/step - loss: 7.6444 - lr: 0.0010
Epoch 5/150
60/61 [============================>.] - ETA: 0s - 

Epoch 36/150
61/61 [==============================] - ETA: 0s - loss: 2.6797
Epoch 36: loss improved from 2.77116 to 2.67969, saving model to nextword1.h5
61/61 [==============================] - 4s 63ms/step - loss: 2.6797 - lr: 0.0010
Epoch 37/150
61/61 [==============================] - ETA: 0s - loss: 2.6384
Epoch 37: loss improved from 2.67969 to 2.63842, saving model to nextword1.h5
61/61 [==============================] - 4s 64ms/step - loss: 2.6384 - lr: 0.0010
Epoch 38/150
61/61 [==============================] - ETA: 0s - loss: 2.6163
Epoch 38: loss improved from 2.63842 to 2.61630, saving model to nextword1.h5
61/61 [==============================] - 4s 65ms/step - loss: 2.6163 - lr: 0.0010
Epoch 39/150
61/61 [==============================] - ETA: 0s - loss: 2.5475
Epoch 39: loss improved from 2.61630 to 2.54746, saving model to nextword1.h5
61/61 [==============================] - 4s 64ms/step - loss: 2.5475 - lr: 0.0010
Epoch 40/150
61/61 [==============================] 

Epoch 71/150
61/61 [==============================] - ETA: 0s - loss: 1.5774
Epoch 71: loss improved from 1.59752 to 1.57741, saving model to nextword1.h5
61/61 [==============================] - 4s 64ms/step - loss: 1.5774 - lr: 0.0010
Epoch 72/150
61/61 [==============================] - ETA: 0s - loss: 1.5482
Epoch 72: loss improved from 1.57741 to 1.54825, saving model to nextword1.h5
61/61 [==============================] - 4s 64ms/step - loss: 1.5482 - lr: 0.0010
Epoch 73/150
61/61 [==============================] - ETA: 0s - loss: 1.5451
Epoch 73: loss improved from 1.54825 to 1.54511, saving model to nextword1.h5
61/61 [==============================] - 4s 64ms/step - loss: 1.5451 - lr: 0.0010
Epoch 74/150
60/61 [============================>.] - ETA: 0s - loss: 1.5086
Epoch 74: loss improved from 1.54511 to 1.51366, saving model to nextword1.h5
61/61 [==============================] - 4s 64ms/step - loss: 1.5137 - lr: 0.0010
Epoch 75/150
61/61 [==============================] 

Epoch 106/150
60/61 [============================>.] - ETA: 0s - loss: 0.7567
Epoch 106: loss improved from 0.76346 to 0.75935, saving model to nextword1.h5
61/61 [==============================] - 4s 65ms/step - loss: 0.7593 - lr: 2.0000e-04
Epoch 107/150
61/61 [==============================] - ETA: 0s - loss: 0.7582
Epoch 107: loss improved from 0.75935 to 0.75824, saving model to nextword1.h5
61/61 [==============================] - 4s 65ms/step - loss: 0.7582 - lr: 2.0000e-04
Epoch 108/150
60/61 [============================>.] - ETA: 0s - loss: 0.7546
Epoch 108: loss improved from 0.75824 to 0.75655, saving model to nextword1.h5
61/61 [==============================] - 4s 64ms/step - loss: 0.7565 - lr: 2.0000e-04
Epoch 109/150
61/61 [==============================] - ETA: 0s - loss: 0.7555
Epoch 109: loss improved from 0.75655 to 0.75550, saving model to nextword1.h5
61/61 [==============================] - 4s 65ms/step - loss: 0.7555 - lr: 2.0000e-04
Epoch 110/150
61/61 [=======

61/61 [==============================] - 4s 64ms/step - loss: 0.6653 - lr: 1.0000e-04
Epoch 141/150
61/61 [==============================] - ETA: 0s - loss: 0.6661
Epoch 141: loss did not improve from 0.66532
61/61 [==============================] - 4s 59ms/step - loss: 0.6661 - lr: 1.0000e-04
Epoch 142/150
60/61 [============================>.] - ETA: 0s - loss: 0.6663
Epoch 142: loss did not improve from 0.66532
61/61 [==============================] - 4s 58ms/step - loss: 0.6660 - lr: 1.0000e-04
Epoch 143/150
61/61 [==============================] - ETA: 0s - loss: 0.6670
Epoch 143: loss did not improve from 0.66532
61/61 [==============================] - 4s 58ms/step - loss: 0.6670 - lr: 1.0000e-04
Epoch 144/150
61/61 [==============================] - ETA: 0s - loss: 0.6645
Epoch 144: loss improved from 0.66532 to 0.66455, saving model to nextword1.h5
61/61 [==============================] - 4s 63ms/step - loss: 0.6645 - lr: 1.0000e-04
Epoch 145/150
61/61 [=======================

## Observation:
### We are able to develop a decent next word prediction model and are able to get a declining loss and an overall decent performance.